### Содержание

#### [Задание 1: Имена](#names)
#### [Задание 2: IP-адреса](#ip)
#### [Задание 3: Оценка уроков](#lessons)

## Задание 1: Имена<a name="names"></a>

### Исходные данные

[names.txt](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/c47a2634-d6cf-4b2b-9229-95fbdbe92cae/names.txt)

В текстовом файле содержится набор имён.

### Задача

Нужно выполнить следующие действия и вывести результат вычислений:

1. Отсортировать все имена в лексикографическом порядке.
2. Посчитать для каждого имени алфавитную сумму – сумму порядковых номеров букв (MAY: 13 + 1 + 25 = 39).
3. Умножить алфавитную сумму каждого имени на порядковый номер имени в отсортированном списке (индексация начинается с 1). Например, если MAY находится на 63 месте в списке, то результат для него будет 63 * 39 = 2457.
4. Сложить произведения из п. 3 для всех имен из файла и получить число.
5. Вывести полученную сумму.

In [1]:
# загружаем нужные библиотеки для обработки данных

import pandas as pd
import numpy as np
import datetime as dt

from collections import Counter

In [2]:
names_df = pd.read_csv('names.txt', sep=',', header=None).T
names_df.head()

,0
0,MARY
1,PATRICIA
2,LINDA
3,BARBARA
4,ELIZABETH


In [3]:
names_df.columns = ['names']  #название колонки
names_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5165 entries, 0 to 5164
Data columns (total 1 columns):
names    5164 non-null object
dtypes: object(1)
memory usage: 80.7+ KB


#### Отсортировать все имена в лексикографическом порядке

In [4]:
names_df = names_df.sort_values(by='names').reset_index(drop=True)
names_df.dropna(inplace=True)  # удалим отсутствующие значения
names_df.tail()

,names
5159,ZORA
5160,ZORAIDA
5161,ZULA
5162,ZULEMA
5163,ZULMA


In [5]:
names_df.duplicated().sum()     # дубликаты, даже если бы и были - удалять не нужно

0

In [6]:
# удалить пробелы
names_df = names_df.replace(r'\s+','',regex=True)

#### Посчитать для каждого имени алфавитную сумму – сумму порядковых номеров букв (MAY: 13 + 1 + 25 = 39).

In [7]:
# переменная для назначения буквам порядковой цифры
num_alphabet = dict([tuple(reversed(i)) for i in enumerate('ABCDEFGHIJKLMNOPQRSTUWVXYZ', 1)]);

# функция сумирования перебора букв(цифр)
def func_name(data):
    s = sum([num_alphabet[i] for i in data])
    return s;

names_df['names_sum'] = list(map(func_name, names_df['names']))   # применение функции
names_df.head()

,names,names_sum
0,AARON,49
1,ABBEY,35
2,ABBIE,19
3,ABBY,30
4,ABDUL,40


In [8]:
names_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5164 entries, 0 to 5163
Data columns (total 2 columns):
names        5164 non-null object
names_sum    5164 non-null int64
dtypes: int64(1), object(1)
memory usage: 121.0+ KB


#### Умножить алфавитную сумму каждого имени на порядковый номер имени в отсортированном списке (индексация начинается с 1). Например, если MAY находится на 63 месте в списке, то результат для него будет 63 * 39 = 2457.

In [9]:
names_df['multiply_names'] = names_df['names_sum'] * (names_df.index + 1)
names_df

,names,names_sum,multiply_names
0,AARON,49,49
1,ABBEY,35,70
2,ABBIE,19,57
3,ABBY,30,120
4,ABDUL,40,200
...,...,...,...
5159,ZORA,60,309600
5160,ZORAIDA,74,381914
5161,ZULA,60,309720
5162,ZULEMA,78,402714


#### Сложить произведения из п. 3 для всех имен из файла и получить число и вывести полученную сумму.

In [10]:
'Сумма всех произведений алфавитных сумм имен на место по индексу {:,}'.format(names_df.multiply_names.sum())

'Сумма всех произведений алфавитных сумм имен на место по индексу 872,141,665'

## Задание 2: IP-адреса<a name="ip"></a>

### Исходные данные

[hits.txt](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/f4f86e5b-f716-478a-9750-5960998f852f/hits.txt)

Файл состоит из строк вида: `<host>\t<ip>\t<page>\n`, где host — корневой домен, ip — IP-адрес, page — «хвост» ссылки.

### Задача

Необходимо вывести 5 IP-адресов, которые встречаются в файле чаще других.

In [11]:
ip_df = pd.read_csv('hits.txt', sep='\t', header=None)
ip_df.head()

,0,1,2
0,https://wikipedia.org,72.110.191.15,/friends/files
1,rambler.ru,142.93.168.247,/universe.php
2,ftp://info.itar-tass.com,90.68.118.69,/login/monitors/static/me
3,http://altavista.com,18.79.197.236,/friends/tasks/universe
4,ftp://tetrika-school.ru,115.26.250.226,/translate/anekdots/my-files/monday


In [12]:
ip_df.columns = ['host', 'ip', 'page']
ip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247036 entries, 0 to 1247035
Data columns (total 3 columns):
host    1247036 non-null object
ip      1247036 non-null object
page    1247036 non-null object
dtypes: object(3)
memory usage: 28.5+ MB


"Пустых" данных нет. Дублкаты искать нет смысла иначе удалим 5 наиболее часто встречающихся ip   

**ТОП=5 наиболее часто стречающихся IP адресов**

In [13]:
ip_df['ip'].value_counts().head()

154.157.157.156    1531
82.146.232.163     1505
226.247.119.128    1494
194.78.107.33      1494
21.143.243.182     1479
Name: ip, dtype: int64

## Задание 3: Оценка уроков<a name="lessons"></a>

*В нашей школе ученики и репетиторы занимаются в специальном онлайн-классе, в котором они могут общаться друг с другом, рисовать на доске, переписываться в чате, обмениваться файлами и решать различные задачки. Информацию о каждом уроке мы записываем в базу данных в таблицу lessons. На каждом таком уроке присутствует один репетитор и один ученик. В конце урока, когда учитель и ученик покидают класс, им предлагается по желанию оценить качество пройденного урока по пятибальной шкале. Оценки мы записываем в таблицу quality.*

### Исходные данные

[tech_quality.zip](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/f35472a4-8bd2-49fa-b69b-a86349b6040b/tech_quality.zip)

В архиве содержится четыре файла с выгрузкой строк из базы.

#### ПОЛЬЗОВАТЕЛИ
Файл **users.txt** хранит информацию о пользователях:

- id – уникальный идентификатор пользователя
- role – указывает, является ли пользователь учеником (pupil) или учителем (tutor)

In [14]:
users_df = pd.read_csv('users.txt')
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745 entries, 0 to 744
Data columns (total 1 columns):
                  id                  | role      745 non-null object
dtypes: object(1)
memory usage: 5.9+ KB


одна колонка с нестандартным названием

In [15]:
users_df.head()

,id | role
0,--------------------------------------+-------
1,e28351f5-4ccb-4549-8647-d43f2b15e7b8 | pupil
2,4df2832a-1d63-4453-9659-43993fc35996 | tutor
3,bb1c0bc8-1212-452b-97a0-439d4a2169e2 | pupil
4,63441abe-c4da-4275-ba26-66f7dbd65dde | tutor


In [16]:
users_df.columns    # название столбца

Index(['                  id                  | role  '], dtype='object')

In [17]:
# разделим колонку на несколько столбцов
users_df = users_df['                  id                  | role  '].str.split(expand=True)
users_df.head()

,0,1,2
0,--------------------------------------+-------,None,None
1,e28351f5-4ccb-4549-8647-d43f2b15e7b8,|,pupil
2,4df2832a-1d63-4453-9659-43993fc35996,|,tutor
3,bb1c0bc8-1212-452b-97a0-439d4a2169e2,|,pupil
4,63441abe-c4da-4275-ba26-66f7dbd65dde,|,tutor


In [18]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745 entries, 0 to 744
Data columns (total 3 columns):
0    745 non-null object
1    744 non-null object
2    743 non-null object
dtypes: object(3)
memory usage: 17.6+ KB


In [19]:
users_df.rename(columns={0: 'user_id', 2: 'role'}, inplace=True)   # переименуем столбцы
users_df.drop(1, axis=1, inplace=True)                             # удалим ненужный столбец
users_df.dropna(inplace=True)                                      # удалим строки не несущие информацию
users_df = users_df.replace(r'\s+','',regex=True)                  # удалить пробелы
users_df.head()                                                    # проверка

,user_id,role
1,e28351f5-4ccb-4549-8647-d43f2b15e7b8,pupil
2,4df2832a-1d63-4453-9659-43993fc35996,tutor
3,bb1c0bc8-1212-452b-97a0-439d4a2169e2,pupil
4,63441abe-c4da-4275-ba26-66f7dbd65dde,tutor
5,a1323b68-c82c-429a-8e2c-65597e648c1a,tutor


In [20]:
# проверим дубликаты
users_df.duplicated().sum()

475

In [21]:
users_df[users_df.duplicated()]          # что за дубликаты

,user_id,role
11,63441abe-c4da-4275-ba26-66f7dbd65dde,tutor
13,43efce48-94b2-4412-857f-223d45969008,tutor
21,30a19496-bdaf-461c-abbc-2709ae520201,tutor
25,ad7199dd-3bf6-4d6d-8ae7-448ecf90a4eb,tutor
29,43efce48-94b2-4412-857f-223d45969008,tutor
...,...,...
736,8476b612-223f-4c86-9b0b-14bc04759233,tutor
737,ab7149fc-b650-4b6b-a368-0581f78276ea,pupil
738,8476b612-223f-4c86-9b0b-14bc04759233,tutor
740,ad7199dd-3bf6-4d6d-8ae7-448ecf90a4eb,tutor


In [22]:
Counter(users_df['user_id'])  # количество упоминаний

Counter({'e28351f5-4ccb-4549-8647-d43f2b15e7b8': 2,
         '4df2832a-1d63-4453-9659-43993fc35996': 15,
         'bb1c0bc8-1212-452b-97a0-439d4a2169e2': 3,
         '63441abe-c4da-4275-ba26-66f7dbd65dde': 23,
         'a1323b68-c82c-429a-8e2c-65597e648c1a': 2,
         'e866fbd8-a46b-4431-9e19-f0966981f6c9': 2,
         '4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2': 1,
         '30a19496-bdaf-461c-abbc-2709ae520201': 7,
         '43efce48-94b2-4412-857f-223d45969008': 33,
         '6daa4b70-1e22-46c8-bcb4-22d452e4add1': 1,
         '2ebe48bd-0d4b-41e3-9f7e-02e006227df9': 3,
         'e4e0a62e-5f9f-4d44-aa3f-e6827a926d9f': 2,
         'ad7199dd-3bf6-4d6d-8ae7-448ecf90a4eb': 25,
         'ab9a1804-b421-43d7-b052-84e15a1d8072': 1,
         '8fe03f08-8581-430c-a590-9888ab36deb3': 24,
         '3c5d30e3-8831-490e-a2a4-63e6ff1c33bb': 6,
         '696c838e-c054-4e9f-a51a-50bf5660f364': 18,
         '621ea7d1-5ffc-494a-a877-dd7d9c75bb79': 4,
         '5ac42f66-8533-46ab-9ccf-11aead34601f': 2,
      

In [23]:
# удаляем дубликаты
users_df.drop_duplicates(inplace=True)
users_df.head()

,user_id,role
1,e28351f5-4ccb-4549-8647-d43f2b15e7b8,pupil
2,4df2832a-1d63-4453-9659-43993fc35996,tutor
3,bb1c0bc8-1212-452b-97a0-439d4a2169e2,pupil
4,63441abe-c4da-4275-ba26-66f7dbd65dde,tutor
5,a1323b68-c82c-429a-8e2c-65597e648c1a,tutor


In [24]:
users_df.shape

(268, 2)

#### УРОКИ
Файл **lessons.txt** содержит следующие поля:

- id – уникальный идентификатор урока
- event_id – идентификатор, связывающий уроки с файлом participants. У нескольких уроков может быть один и тот же event_id
- subject – предмет урока, строка
- scheduled_time – время начала урока, в формате ГГГГ-ММ-ДД чч:мм:сс (иногда есть еще миллисекунды). Время указано в UTC

In [25]:
lessons_df = pd.read_csv('lessons.txt', sep='|')
lessons_df

,id,event_id,subject,scheduled_time
0,--------------------------------------+-------...,NaN,NaN,NaN
1,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114.0,phys,2020-01-19 12:00:00
2,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,51568.0,it,2020-01-19 13:00:00
3,62e1a078-33de-47c1-99d2-845b1daca56f,52790.0,hist,2020-01-19 13:00:00
4,00fc6685-f53a-49bb-b960-5e0042fd3852,51341.0,phys,2020-01-17 12:00:00
...,...,...,...,...
375,056c3d91-dce7-4071-91d3-1de816ee63a3,51001.0,hist,2020-01-17 12:45:00
376,c64a1daf-d7f5-4c8c-ac9f-df634d77327b,50304.0,bio,2020-01-11 11:00:00
377,aa7702bc-5271-469c-a1c2-b7e814fd7e20,49880.0,bio,2020-01-16 15:00:00
378,8643fa00-217a-42ae-a27d-e6f47c0501f1,55449.0,bio,2020-01-19 12:15:00


In [26]:
lessons_df.dropna(inplace=True)                                # удалим строки не несущие информацию
lessons_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378 entries, 1 to 378
Data columns (total 4 columns):
                  id                      378 non-null object
 event_id                                 378 non-null float64
 subject                                  378 non-null object
       scheduled_time                     378 non-null object
dtypes: float64(1), object(3)
memory usage: 14.8+ KB


Пустых значений нет, нужно изменить названия столбцов (убрать пробелы), изменить типы данных: для event_id - целочисленнный формат, для времени - формат дат по московскому времени.

In [27]:
lessons_df.columns = ['lesson_id', 'event_id', 'subject', 'scheduled_time']                   # название столбцов
lessons_df['event_id'] = lessons_df['event_id'].astype('int')                                 # целочисленные данные 

In [28]:
lessons_df['scheduled_time'] = pd.to_datetime(lessons_df['scheduled_time']).dt.round('1S')    # округляем до секунды
lessons_df.head()

,lesson_id,event_id,subject,scheduled_time
1,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 12:00:00
2,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,51568,it,2020-01-19 13:00:00
3,62e1a078-33de-47c1-99d2-845b1daca56f,52790,hist,2020-01-19 13:00:00
4,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00
5,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00


In [29]:
# время по Москве = UTC+3, оставляем информацию о дне проведения урока
lessons_df['moscow_date'] = (lessons_df['scheduled_time'] + dt.timedelta(hours=3)).astype('datetime64[D]')  
lessons_df.head()

,lesson_id,event_id,subject,scheduled_time,moscow_date
1,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 12:00:00,2020-01-19
2,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,51568,it,2020-01-19 13:00:00,2020-01-19
3,62e1a078-33de-47c1-99d2-845b1daca56f,52790,hist,2020-01-19 13:00:00,2020-01-19
4,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,2020-01-17
5,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00,2020-01-19


In [30]:
lessons_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378 entries, 1 to 378
Data columns (total 5 columns):
lesson_id         378 non-null object
event_id          378 non-null int32
subject           378 non-null object
scheduled_time    378 non-null datetime64[ns]
moscow_date       378 non-null datetime64[ns]
dtypes: datetime64[ns](2), int32(1), object(2)
memory usage: 16.2+ KB


In [31]:
lessons_df.duplicated().sum()

0

In [32]:
lessons_df.subject.unique()

array([' phys    ', ' it      ', ' hist    ', ' bio     '], dtype=object)

In [33]:
lessons_df.lesson_id.unique()

array([' 1e7bb408-cfef-4a9f-8328-351c9483a64c ',
       ' 6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a ',
       ' 62e1a078-33de-47c1-99d2-845b1daca56f ',
       ' 00fc6685-f53a-49bb-b960-5e0042fd3852 ',
       ' 4cadf623-82e6-422f-a342-acf978302fb2 ',
       ' f3f15fee-ad91-429c-8b1e-faeb42f78f58 ',
       ' e597a79a-3f68-4d15-bc77-c7000bcf8e52 ',
       ' 44e2e74c-8481-41a2-bd64-f6af688169b7 ',
       ' ea6f0bf3-bcfc-4555-9b16-cc2cfe722e17 ',
       ' 7d56b43a-d137-446d-8570-8d7d5ccbaee9 ',
       ' 38a8b0ea-ff54-42cf-ab5b-742700ffcd0a ',
       ' b1a122d7-b1c3-43c9-86e1-ec5e40456a3b ',
       ' d8816fa1-dd30-4a4b-8bd1-32285dec279f ',
       ' ec953406-b4a3-416a-b1b6-4c4853995047 ',
       ' 952674d6-5d00-4797-95f7-6686014a6b65 ',
       ' 7c60dff9-7145-4a89-8310-e5ad951f7758 ',
       ' 84adb28c-182d-4c9f-b5ef-4f8138368d85 ',
       ' 36e05c76-2e4d-46d8-bd3b-fdcfa98b962b ',
       ' 59e1a7df-46bf-4415-a711-67912df0f64a ',
       ' 4fd4f095-a793-4375-82e8-7250dd5036b0 ',
       ' cf2dcbaf-06

In [34]:
lessons_df.lesson_id = lessons_df.lesson_id.replace(r'\s+','',regex=True)           # удаляем пробелы
lessons_df.subject = lessons_df.subject.replace(r'\s+','',regex=True)          

#### УЧАСТНИКИ
Файл **participants.txt** позволяет связать урок с его участниками. Он содержит следующие поля:

- user_id – идентификатор пользователя (указывает на запись в файле users.txt)
- event_id – идентификатор, связывающий урок с участником. Чтобы понять, какие пользователи были на уроке Х, нужно найти в файле participants.txt строки, у которых event_id совпадает с event_id урока Х.

In [35]:
participants_df = pd.read_csv('participants.txt', sep='|')
participants_df

,event_id,user_id
0,----------+--------------------------------------,NaN
1,38114,e28351f5-4ccb-4549-8647-d43f2b15e7b8
2,38114,4df2832a-1d63-4453-9659-43993fc35996
3,51568,bb1c0bc8-1212-452b-97a0-439d4a2169e2
4,51568,63441abe-c4da-4275-ba26-66f7dbd65dde
...,...,...
740,49880,8476b612-223f-4c86-9b0b-14bc04759233
741,49880,582e6abb-dd9a-42fa-b441-455eac28327e
742,55449,ad7199dd-3bf6-4d6d-8ae7-448ecf90a4eb
743,55449,d3364cf3-46cc-4e64-ad11-70a069f7c049


In [36]:
participants_df.dropna(inplace=True)                                         # удалим строки не несущие информацию
participants_df.columns = ['event_id', 'user_id']                            # название столбцов
participants_df['event_id'] = participants_df['event_id'].astype('int')      # изменим тип данных
participants_df = participants_df.replace(r'\s+','',regex=True)              # удаляем пробелы
participants_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 743 entries, 1 to 743
Data columns (total 2 columns):
event_id    743 non-null int64
user_id     743 non-null object
dtypes: int64(1), object(1)
memory usage: 17.4+ KB


In [37]:
participants_df.duplicated().sum()

98

так как в базе связывается однозначно уникальный урок с уникальным участником - можем удалить дубликаты из таблицы.

In [38]:
participants_df.drop_duplicates(inplace=True)                                # удалим дубликаты
participants_df.shape

(645, 2)

#### ОЦЕНКИ
Файл **quality.txt** содержит следующие поля:

- lesson_id – идентификатор урока, указывающий на запись в таблице lessons.txt. У нескольких строчек из этого файла может быть один и тот же lesson_id, потому что оценок за урок может быть несколько (в случае, когда и ученик, и репетитор оценили качество урока)
- tech_quality – оценка качества урока. Это число от 1 до 5. Иногда его может не быть, если пользователь не выставил оценку.

In [39]:
quality_df = pd.read_csv('quality.txt', sep='|')
quality_df

,lesson_id,tech_quality
0,--------------------------------------+-------...,NaN
1,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,5
2,62e1a078-33de-47c1-99d2-845b1daca56f,5
3,62e1a078-33de-47c1-99d2-845b1daca56f,5
4,00fc6685-f53a-49bb-b960-5e0042fd3852,5
...,...,...
362,aa7702bc-5271-469c-a1c2-b7e814fd7e20,5
363,8643fa00-217a-42ae-a27d-e6f47c0501f1,5
364,7a15af58-39d0-4207-bfdf-3fa176b7fb4e,5
365,7a15af58-39d0-4207-bfdf-3fa176b7fb4e,5


In [40]:
quality_df.dropna(inplace=True)                                # удалим строки не несущие информацию
quality_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 1 to 365
Data columns (total 2 columns):
              lesson_id                   365 non-null object
 tech_quality                             365 non-null object
dtypes: object(2)
memory usage: 8.6+ KB


In [41]:
quality_df.columns = ['lesson_id', 'tech_quality']                                # название столбцов
quality_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 1 to 365
Data columns (total 2 columns):
lesson_id       365 non-null object
tech_quality    365 non-null object
dtypes: object(2)
memory usage: 8.6+ KB


In [42]:
quality_df['tech_quality'].unique()        # проверим в каком состоянии оценки

array(['            5', '            2', '            4', '             ',
       '            3', '            1'], dtype=object)

In [43]:
# уберем пробелы, заменим тип данных
quality_df['tech_quality'] = quality_df['tech_quality'].apply(pd.to_numeric, errors='coerce')
quality_df['tech_quality'].unique()

array([ 5.,  2.,  4., nan,  3.,  1.])

In [44]:
quality_df = quality_df.replace(r'\s+','',regex=True)              # удаляем пробелы
quality_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 1 to 365
Data columns (total 2 columns):
lesson_id       365 non-null object
tech_quality    349 non-null float64
dtypes: float64(1), object(1)
memory usage: 8.6+ KB


In [45]:
quality_df.head()

,lesson_id,tech_quality
1,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,5.0
2,62e1a078-33de-47c1-99d2-845b1daca56f,5.0
3,62e1a078-33de-47c1-99d2-845b1daca56f,5.0
4,00fc6685-f53a-49bb-b960-5e0042fd3852,5.0
5,00fc6685-f53a-49bb-b960-5e0042fd3852,5.0


In [46]:
quality_df.describe()

,tech_quality
count,349.000000
mean,4.793696
std,0.650257
min,1.000000
25%,5.000000
50%,5.000000
75%,5.000000
max,5.000000


исходя из условия подачи данных - дубликаты не нужно искать

### Задача

Нужно найти, какой из репетиторов получал самую низкую среднюю оценку качества после уроков по физике за каждый день, и вывести эту оценку.

Примечания:

- Необходимо учитывать оценки и учителей, и учеников.
- Нужно учитывать только тех учителей, за уроки по физике которых в этот день стоит хотя бы одна оценка.
- Начало урока для решения задачи должно учитываться по московскому времени.
- При расчёте среднего значения не нужно учитывать ситуации, когда пользователь не выставил оценку.
- Если у нескольких учителей после расчёта совпадают средние значения, можно вывести любого из них.

Решение должно выглядеть примерно так:

[result](https://www.notion.so/98bcad0395144058ac50c53ae07590aa)

Из базы пользователей выберем только учителей

In [47]:
users_df.head()

,user_id,role
1,e28351f5-4ccb-4549-8647-d43f2b15e7b8,pupil
2,4df2832a-1d63-4453-9659-43993fc35996,tutor
3,bb1c0bc8-1212-452b-97a0-439d4a2169e2,pupil
4,63441abe-c4da-4275-ba26-66f7dbd65dde,tutor
5,a1323b68-c82c-429a-8e2c-65597e648c1a,tutor


In [48]:
users_df.shape

(268, 2)

In [49]:
tutor = users_df['role'] == 'tutor'
users_tutor = users_df.query('@tutor')    # база id только преподователей
users_tutor

,user_id,role
2,4df2832a-1d63-4453-9659-43993fc35996,tutor
4,63441abe-c4da-4275-ba26-66f7dbd65dde,tutor
5,a1323b68-c82c-429a-8e2c-65597e648c1a,tutor
8,30a19496-bdaf-461c-abbc-2709ae520201,tutor
9,43efce48-94b2-4412-857f-223d45969008,tutor
...,...,...
543,1c39e78f-6ded-4b2e-83d6-036ca34ecfdc,tutor
566,d754610c-c4ac-46f5-9b02-8067e0f8a01d,tutor
600,4758da03-f56d-4cf6-a58d-17dd3d7d8445,tutor
627,c7123879-75e7-4957-b809-f2f684499b67,tutor


In [50]:
lessons_df.head()

,lesson_id,event_id,subject,scheduled_time,moscow_date
1,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 12:00:00,2020-01-19
2,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,51568,it,2020-01-19 13:00:00,2020-01-19
3,62e1a078-33de-47c1-99d2-845b1daca56f,52790,hist,2020-01-19 13:00:00,2020-01-19
4,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,2020-01-17
5,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00,2020-01-19


In [51]:
lessons_df.shape

(378, 5)

In [52]:
participants_df.head()

,event_id,user_id
1,38114,e28351f5-4ccb-4549-8647-d43f2b15e7b8
2,38114,4df2832a-1d63-4453-9659-43993fc35996
3,51568,bb1c0bc8-1212-452b-97a0-439d4a2169e2
4,51568,63441abe-c4da-4275-ba26-66f7dbd65dde
5,52790,a1323b68-c82c-429a-8e2c-65597e648c1a


In [53]:
participants_df.shape

(645, 2)

Составим базу событий с участием только преподователей - какие уроки вели преподователи

In [54]:
tutor_event = pd.merge(users_tutor, participants_df, on='user_id')
tutor_event.columns = ['tutor_id', 'role', 'event_id']
tutor_event

,tutor_id,role,event_id
0,4df2832a-1d63-4453-9659-43993fc35996,tutor,38114
1,4df2832a-1d63-4453-9659-43993fc35996,tutor,38111
2,4df2832a-1d63-4453-9659-43993fc35996,tutor,56195
3,4df2832a-1d63-4453-9659-43993fc35996,tutor,50880
4,4df2832a-1d63-4453-9659-43993fc35996,tutor,51450
...,...,...,...
324,1c39e78f-6ded-4b2e-83d6-036ca34ecfdc,tutor,55480
325,d754610c-c4ac-46f5-9b02-8067e0f8a01d,tutor,55441
326,4758da03-f56d-4cf6-a58d-17dd3d7d8445,tutor,55572
327,c7123879-75e7-4957-b809-f2f684499b67,tutor,55660


In [55]:
phys = lessons_df['subject'] == 'phys'                     # только уроки по физике
lessons_phys = lessons_df.query('@phys')
lessons_phys

,lesson_id,event_id,subject,scheduled_time,moscow_date
1,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 12:00:00,2020-01-19
4,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,2020-01-17
5,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00,2020-01-19
7,e597a79a-3f68-4d15-bc77-c7000bcf8e52,53026,phys,2020-01-19 15:00:00,2020-01-19
9,ea6f0bf3-bcfc-4555-9b16-cc2cfe722e17,51645,phys,2020-01-19 14:00:00,2020-01-19
...,...,...,...,...,...
367,e90f529b-1061-4efd-9bba-f73ecee2ef27,55490,phys,2020-01-19 08:00:00,2020-01-19
368,f0002160-e57e-4e90-ac7e-c8fcdbbae26f,41963,phys,2020-01-19 09:00:00,2020-01-19
369,8a1bac54-c5c6-47c9-8fdb-36d01384eaae,55881,phys,2020-01-19 15:00:00,2020-01-19
370,457accf4-bd99-4b9d-b540-ef200e65afbd,49007,phys,2020-01-19 05:00:00,2020-01-19


Объединяем базу уроков по физике и оценок за урок

In [56]:
lessons_phys_quality = pd.merge(lessons_phys, quality_df, on='lesson_id', how='left')
lessons_phys_quality

,lesson_id,event_id,subject,scheduled_time,moscow_date,tech_quality
0,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 12:00:00,2020-01-19,NaN
1,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,2020-01-17,5.0
2,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,2020-01-17,5.0
3,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00,2020-01-19,5.0
4,e597a79a-3f68-4d15-bc77-c7000bcf8e52,53026,phys,2020-01-19 15:00:00,2020-01-19,5.0
...,...,...,...,...,...,...
243,8a1bac54-c5c6-47c9-8fdb-36d01384eaae,55881,phys,2020-01-19 15:00:00,2020-01-19,NaN
244,457accf4-bd99-4b9d-b540-ef200e65afbd,49007,phys,2020-01-19 05:00:00,2020-01-19,5.0
245,457accf4-bd99-4b9d-b540-ef200e65afbd,49007,phys,2020-01-19 05:00:00,2020-01-19,4.0
246,403707fd-b110-4216-b678-6dbc92eb6fad,41964,phys,2020-01-19 11:00:00,2020-01-19,NaN


*За какие-то уроки оценки не ставились*

**Объединяем базу уроков по физике с оценками за урок и базу учителей**

In [57]:
lessons_phys_quality = lessons_phys_quality.merge(tutor_event, on='event_id', how='left')
lessons_phys_quality

,lesson_id,event_id,subject,scheduled_time,moscow_date,tech_quality,tutor_id,role
0,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 12:00:00,2020-01-19,NaN,4df2832a-1d63-4453-9659-43993fc35996,tutor
1,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,2020-01-17,5.0,30a19496-bdaf-461c-abbc-2709ae520201,tutor
2,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,2020-01-17,5.0,30a19496-bdaf-461c-abbc-2709ae520201,tutor
3,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00,2020-01-19,5.0,43efce48-94b2-4412-857f-223d45969008,tutor
4,e597a79a-3f68-4d15-bc77-c7000bcf8e52,53026,phys,2020-01-19 15:00:00,2020-01-19,5.0,43efce48-94b2-4412-857f-223d45969008,tutor
...,...,...,...,...,...,...,...,...
243,8a1bac54-c5c6-47c9-8fdb-36d01384eaae,55881,phys,2020-01-19 15:00:00,2020-01-19,NaN,c6718d0e-976c-4d6c-b0e0-32c770776567,tutor
244,457accf4-bd99-4b9d-b540-ef200e65afbd,49007,phys,2020-01-19 05:00:00,2020-01-19,5.0,696c838e-c054-4e9f-a51a-50bf5660f364,tutor
245,457accf4-bd99-4b9d-b540-ef200e65afbd,49007,phys,2020-01-19 05:00:00,2020-01-19,4.0,696c838e-c054-4e9f-a51a-50bf5660f364,tutor
246,403707fd-b110-4216-b678-6dbc92eb6fad,41964,phys,2020-01-19 11:00:00,2020-01-19,NaN,8fe03f08-8581-430c-a590-9888ab36deb3,tutor


**Считаем среднюю оценку за уроки для каждого преподователя в определенный день**

In [58]:
mean_phys = lessons_phys_quality.groupby(['moscow_date', 'tutor_id']).agg({'tech_quality': 'mean'}).reset_index()
mean_phys.columns = ['moscow_date', 'tutor_id', 'average_score']
mean_phys['average_score'] = mean_phys['average_score'].round(2)        # округляем до двух знаков
mean_phys.sort_values(by='average_score').head()                        # сортировка по возрастанию 

,moscow_date,tutor_id,average_score
46,2020-01-16,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,4.00
37,2020-01-14,c6718d0e-976c-4d6c-b0e0-32c770776567,4.00
71,2020-01-18,43efce48-94b2-4412-857f-223d45969008,4.25
7,2020-01-11,8fe03f08-8581-430c-a590-9888ab36deb3,4.43
92,2020-01-20,43efce48-94b2-4412-857f-223d45969008,4.50


**Минимальная средняя оценка за урок в определенный день**

In [59]:
mean_phys_min = mean_phys.groupby(['moscow_date'])['average_score'].min().reset_index()
mean_phys_min

,moscow_date,average_score
0,2020-01-11,4.43
1,2020-01-12,4.89
2,2020-01-13,5.00
3,2020-01-14,4.00
4,2020-01-15,5.00
5,2020-01-16,4.00
6,2020-01-17,4.50
7,2020-01-18,4.25
8,2020-01-19,4.50
9,2020-01-20,4.50


**Определяем преподователей с минимальными средними оценками за урок в определенный день**

In [60]:
table = pd.merge(mean_phys_min, mean_phys, how='left', on=['moscow_date','average_score'])
cols = ['moscow_date', 'tutor_id', 'average_score']
table = table[cols]
table

,moscow_date,tutor_id,average_score
0,2020-01-11,8fe03f08-8581-430c-a590-9888ab36deb3,4.43
1,2020-01-12,696c838e-c054-4e9f-a51a-50bf5660f364,4.89
2,2020-01-13,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,5.00
3,2020-01-13,30a19496-bdaf-461c-abbc-2709ae520201,5.00
4,2020-01-13,696c838e-c054-4e9f-a51a-50bf5660f364,5.00
5,2020-01-13,b37ccae8-fc31-4ad8-8f55-ca855b23fbf6,5.00
6,2020-01-13,be676776-8366-4c71-8a35-d58014806eb5,5.00
7,2020-01-14,c6718d0e-976c-4d6c-b0e0-32c770776567,4.00
8,2020-01-15,603b8641-c6f6-4d89-ac89-88e50d45aa0d,5.00
9,2020-01-15,696c838e-c054-4e9f-a51a-50bf5660f364,5.00
